## To Do
* Compare to randomly generated parties
* Add more modes (Normal, Typhoon, Valcano, Duplicates, 750, no750, 375, Classic?, Onion?)
* Try NN based embeddings
* Make something available on the web (1st iteration, maybe with Google Forms or Sheets)
* Ping Discord Channel for feedback

In [ ]:
import numpy as np
import pandas as pd
from numpy.linalg import norm
from random import randint, seed
from time import time

from data import load_data, load_party_embeddings, save_party_embeddings
from embeddings import calculate_party_embeddings
from experiment import run_trials
from generate_possible_jobs import generate_possible_jobs
from select_parties import select_parties_by_embeddings

seed(a=None)  # Initialize the random seed

In [ ]:
# Set some parameters
equip_factor = 0.5
should_calculate_party_embeddings = False
party_embeddings_filename = f"data/embeddings_meteor_duplicates_eq{equip_factor}.csv"

In [ ]:
# Load data for each job
df_jobs, stat_cols = load_data("data_jobs/job_data_embeddings.csv")

In [ ]:
# Generate all possible parties
start = time()
valid_parties = generate_possible_jobs(style="Meteor", df_jobs=df_jobs, stat_cols=stat_cols)
stop = time()
print(f"Needed {stop-start} seconds.")

In [ ]:
# Calculate or load the party embeddings
start = time()
if should_calculate_party_embeddings:
    print("Calculating party embeddings...")
    valid_parties_embeddings = calculate_party_embeddings(valid_parties, df_jobs, stat_cols, equip_factor)
    save_party_embeddings(party_embeddings_filename, valid_parties_embeddings)
else:
    print("Loading party embeddings...")
    valid_parties_embeddings = load_party_embeddings(party_embeddings_filename)
stop = time()
print(f"Needed {stop-start} seconds.")

In [ ]:
# Try selecting jobs
start = time()
selected_parties = select_parties_by_embeddings(valid_parties_embeddings, num_parties=5, eps=10.0)
stop = time()
print(f"Needed {stop-start} seconds.")

for idx, party_tuple in enumerate(selected_parties):
    print(f"{idx} {party_tuple[0]}")

In [ ]:
# Try some trials
trials = run_trials(valid_parties_embeddings, num_parties=5, num_trials=5, eps=3.0)
print(trials)